In [8]:
library(cowplot)
library(datateachr)
library(digest)
library(infer)
library(repr)
library(taxyvr)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [14]:
wine_quality <- read.csv('winequality-red.csv', sep = ";") |>
    select(total.sulfur.dioxide, quality) |>
    group_by(quality) |>
    summarize(mean = mean(total.sulfur.dioxide))
wine_quality

quality,mean
<int>,<dbl>
3,24.90000
4,36.24528
5,56.51395
6,40.86991
7,35.02010
8,33.44444


Introduction
- Why sulfur dioxide
- Effect of ...
- Research Question (why or why not)
- Reason and supoortie articles
- quality rating
- Database, resource, reliability, author
- hypothesis testing (null, alternative, signficance level)
- bootstrap + confidence interval

research question:
Is there a significant difference in total sulphur dioxide levels between different quality rating? 

methods:
- bootstrap
- mean -- group by different samples
- multiple hypothesis test 
- confidence interval
- type 1 type 2 error analysis